In [1]:
import os
data_dir = "sentiment"
for data_ver in ["orig", "new", "combined"]:
    for data_split in ["train", "dev", "test"]:
        data_path = os.path.join(data_dir, data_ver, data_split + '.tsv')
        with open(data_path, 'r') as fp:
            data = fp.readlines()
        
        print("{}: {}".format(data_path, len(data)))
    print()

sentiment/orig/train.tsv: 1708
sentiment/orig/dev.tsv: 246
sentiment/orig/test.tsv: 489

sentiment/new/train.tsv: 1708
sentiment/new/dev.tsv: 246
sentiment/new/test.tsv: 489

sentiment/combined/train.tsv: 3415
sentiment/combined/dev.tsv: 491
sentiment/combined/test.tsv: 977



In [2]:
orig_train = open("./sentiment/orig/train.tsv", 'r').readlines()[1:]
orig_dev = open("./sentiment/orig/dev.tsv", 'r').readlines()[1:]
orig_test = open("./sentiment/orig/test.tsv", 'r').readlines()[1:]

new_train = open("./sentiment/new/train.tsv", 'r').readlines()[1:]
new_dev = open("./sentiment/new/dev.tsv", 'r').readlines()[1:]
new_test = open("./sentiment/new/test.tsv", 'r').readlines()[1:]

In [3]:
from tqdm.notebook import tqdm

def get_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def pairing(orig_samples, new_samples):
    paired_samples = []
    for orig_sample in tqdm(orig_samples):
        orig_sample = orig_sample.split('\t')
        cf_sample = ['negative', ""]
        for new_sample in new_samples[0:]:
            new_sample = new_sample.split('\t')
            if abs(len(orig_sample[1].split()) - len(new_sample[1].split())) <= 10 and \
                get_sim(orig_sample[1], new_sample[1]) > get_sim(orig_sample[1], cf_sample[1]):
                cf_sample = new_sample
        paired_samples.append((orig_sample, cf_sample))
    return paired_samples

paired_train = pairing(orig_train, new_train)
paired_dev = pairing(orig_dev, new_dev)
paired_test = pairing(orig_test, new_test)

In [4]:
import pickle

with open("paired_train.pickle", "wb") as fp:
    pickle.dump(paired_train, fp)
    
with open("paired_dev.pickle", "wb") as fp:
    pickle.dump(paired_dev, fp)
    
with open("paired_test.pickle", "wb") as fp:
    pickle.dump(paired_test, fp)

In [5]:
from nltk.tokenize import word_tokenize

In [6]:
import random
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
        
def visualize(words, masks):
    fig, ax = plt.subplots(figsize=(len(words), 1))
    plt.rc('xtick', labelsize=16)
    heatmap = sn.heatmap([masks], xticklabels=words, yticklabels=False, square=True, \
                         linewidths=0.1, cmap='coolwarm', center=0.5, vmin=0, vmax=1)
    plt.xticks(rotation=45)
    plt.show()

def mask_causal_words(orig_tokens, cf_tokens):
    orig_causal_mask = [0 for _ in range(len(orig_tokens))]
    for orig_idx in range(len(orig_tokens)):
        is_causal = 1
        for cf_idx in range(len(cf_tokens)):
            if orig_tokens[orig_idx] == cf_tokens[cf_idx]:
                is_causal = 0
                break

        if is_causal:
            orig_causal_mask[orig_idx] = 1
    
    return orig_causal_mask
    
def build_causal_mask(paired, sampling_ratio, augment_ratio):
    triplets = []
    for orig_sample, cf_sample in tqdm(paired):
        orig_tokens = word_tokenize(orig_sample[1])
        cf_tokens = word_tokenize(cf_sample[1])
    
        orig_causal_mask = mask_causal_words(orig_tokens, cf_tokens)
        # visualize(orig_tokens, orig_causal_mask)
        # cf_causal_mask = mask_causal_words(cf_tokens, orig_tokens)
        # visualize(cf_tokens, cf_causal_mask)

        if 1 not in orig_causal_mask:
            # print(orig_sample[1], cf_sample[1])
            continue
        
        for _ in range(augment_ratio):
            # 모든 causal 단어를 mask, 모든 non-causal 단어를 mask
            if sampling_ratio is None:
                causal_masked_tokens = [orig_tokens[i] if orig_causal_mask[i] == 0 else "[MASK]" for i in range(len(orig_tokens))]
                noncausal_masked_tokens = [orig_tokens[i] if orig_causal_mask[i] == 1 else "[MASK]" for i in range(len(orig_tokens))]

            # sampling_ratio 갯수 (int) 만큼의 단어를 mask
            elif type(sampling_ratio) == int:
                causal_indices = np.where(np.array(orig_causal_mask) == 1)[0]
                noncausal_indices = np.where(np.array(orig_causal_mask) == 0)[0]

                # print(causal_indices)

                causal_mask_indices = np.random.choice(causal_indices, sampling_ratio)                    
                noncausal_mask_indices = np.random.choice(noncausal_indices, sampling_ratio)

                causal_masked_tokens = [orig_tokens[i] if i not in causal_mask_indices else "[MASK]" for i in range(len(orig_tokens))]
                noncausal_masked_tokens = [orig_tokens[i] if i not in noncausal_mask_indices else "[MASK]" for i in range(len(orig_tokens))]
            
            # sampling_ratio 비율 (%) 만큼의 단어를 mask
            else:
                pass
                
            causal_masked_sample = " ".join(causal_masked_tokens)
            noncausal_masked_sample = " ".join(noncausal_masked_tokens)

            triplets.append((orig_sample[0], orig_sample[1], causal_masked_sample, noncausal_masked_sample))
        
    return triplets

import pickle

# sampling_ratio = 2
# augment_ratio = 1
for sampling_ratio in [2, 3, 4, 5]:
    for augment_ratio in [1, 2, 3]:
        triplets_train = build_causal_mask(paired_train, sampling_ratio=sampling_ratio, augment_ratio=augment_ratio)
        with open("triplets_sampling{}_augmenting{}_train.pickle".format(sampling_ratio, augment_ratio), "wb") as fp:
            pickle.dump(triplets_train, fp)
        
# triplets_dev = build_causal_mask(paired_dev, sampling_ratio=sampling_ratio, augment_ratio=augment_ratio)
# triplets_test = build_causal_mask(paired_test, sampling_ratio=sampling_ratio, augment_ratio=augment_ratio)

In [7]:

    
# with open("triplets_sampling{}_augmenting{}_dev.pickle".format(sampling_ratio, augment_ratio), "wb") as fp:
#     pickle.dump(triplets_dev, fp)
    
# with open("triplets_sampling{}_augmenting{}_test.pickle".format(sampling_ratio, augment_ratio), "wb") as fp:
#     pickle.dump(triplets_test, fp)